# Kubernetes Pods

We've seen already how we can interrogate our cluster server using the kubectl tool, obtaining information about objects such as some system pods, in the system.

Now let's create our own pods using kubectl.

Normally we would not create pods directly using command-line arguments, as we will do here.

We would create objects in a repeatable, declarative way from yaml files, but we will do that later.

The below ```kubectl run``` command will create 2 pods (replicas), each running just 1 container from the k8s-demo image.

Note that we can specify labels on any kubernetes objects, using the -l option followd by key=value pairs, e.g.
    -l "key1=val1,key2=val2"
    
These labels are extremely useful allowing us to select a group of objects using a label selector.

Labels might be used to mark components of the cluster as dev, test or production for example.

Or they might be used to signal hardware specifics of a node, to distinguish physical nodes with SSD disks, or GPU which could then be prioritized for particular types of processing which might be disk i/o or CPU intensive.


In [1]:
echo; echo "-- initial kubectl get all"
kubectl get all

echo; echo "-- kubectl run k8s0demo"
kubectl run k8s-demo -l "app=k8s-demo" --image=mjbright/k8s-demo:1 --replicas=2 --port=8080

echo; echo "-- second kubectl get all"
kubectl get all

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   5h
deployment "k8s-demo" created
NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         0         1s

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            0           1s

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         0         1s

NAME                           READY     STATUS              RESTARTS   AGE
po/k8s-demo-7bc74f7946-8nmlb   0/1       ContainerCreating   0          1s
po/k8s-demo-7bc74f7946-mmc98   0/1       ContainerCreating   0          1s

NAME             TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
svc/kubernetes   ClusterIP   1

In the above output we see that the only running resource initially was an internal service.

Once we perform '''kubectl run'' this spawns a **deployment** called *k8s-demo*, which creates a corrsponding **replicaset** which in turn spawns 2 **pods**.

As we performed the '''kubectl get all''' immediately after the run, we see the status before the pod containers have been created ... in fact there is a delay whilst the images are downloaded (for a first run) and pods instantiated.

Re-running '''kubectl get all''' shows the pods as *running* now.

In [64]:
kubectl get all

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           9h

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         9h

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
deploy/k8s-demo   2         2         2            2           9h

NAME                     DESIRED   CURRENT   READY     AGE
rs/k8s-demo-7bc74f7946   2         2         2         9h

NAME                           READY     STATUS    RESTARTS   AGE
po/k8s-demo-7bc74f7946-8nmlb   1/1       Running   0          9h
po/k8s-demo-7bc74f7946-mmc98   1/1       Running   0          9h

NAME                   TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)          AGE
svc/k8s-demo-service   NodePort    10.101.31.125   <none>        8080:31991/TCP   8m
svc/kubernetes         ClusterIP   10.96.0.1       <none>        443/TCP          14h


... with more information, using option '*-o wide*'

In [63]:
kubectl get all -o wide

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   2         2         2            2           9h        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-7bc74f7946   2         2         2         9h        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=3673093502

NAME              DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
deploy/k8s-demo   2         2         2            2           9h        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo

NAME                     DESIRED   CURRENT   READY     AGE       CONTAINERS   IMAGES                SELECTOR
rs/k8s-demo-7bc74f7946   2         2         2         9h        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo,pod-template-hash=3673093502

NAME              

## Creating resources using yaml specifications

### Obtaining yaml as output

Using the option '-o yaml' to any 'kubectl get' command we can obtain a yaml definition, so instead of

In [65]:
kubectl get deploy/k8s-demo -o wide

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE       CONTAINERS   IMAGES                SELECTOR
k8s-demo   2         2         2            2           9h        k8s-demo     mjbright/k8s-demo:1   app=k8s-demo


we get ...

In [66]:
kubectl get deploy/k8s-demo -o yaml

apiVersion: extensions/v1beta1
kind: Deployment
metadata:
  annotations:
    deployment.kubernetes.io/revision: "1"
  creationTimestamp: 2018-01-18T21:00:50Z
  generation: 1
  labels:
    app: k8s-demo
  name: k8s-demo
  namespace: default
  resourceVersion: "27338"
  selfLink: /apis/extensions/v1beta1/namespaces/default/deployments/k8s-demo
  uid: aa003146-fc92-11e7-8b3e-02420ac00002
spec:
  replicas: 2
  selector:
    matchLabels:
      app: k8s-demo
  strategy:
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
    type: RollingUpdate
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: k8s-demo
    spec:
      containers:
      - image: mjbright/k8s-demo:1
        imagePullPolicy: IfNotPresent
        name: k8s-demo
        ports:
        - containerPort: 8080
          protocol: TCP
        resources: {}
        terminationMessagePath: /dev/termination-log
        terminationMessagePolicy: File
      dnsPolicy: ClusterFirst
      restartPoli

## Accessing our application

To access our application pods we should expose them as a service.
This would have the advantage of being able to address a virtual ip address for our service as pods themselves come and go (so we don't need to determine their address), and also allows load balancing between pods.

### Accessing our application - directly accessing Pods

But there are other more basic ways that we will investigate first.

If we perform '''kubectl get pods -o wide''' we will see the cluster ip of our pods, enabling us to access directly to the pods.

In [67]:
kubectl get pods -o wide

NAME                        READY     STATUS    RESTARTS   AGE       IP           NODE
k8s-demo-7bc74f7946-8nmlb   1/1       Running   0          9h        10.192.3.1   kube-node-2
k8s-demo-7bc74f7946-mmc98   1/1       Running   0          9h        10.192.2.1   kube-node-1


In [68]:
POD1=$(kubectl get pods -o wide --no-headers=true | head -1 | awk '{print $1;}')
POD2=$(kubectl get pods -o wide --no-headers=true | tail -1 | awk '{print $1;}')
POD1_IP=$(kubectl get pods -o wide --no-headers=true | head -1 | awk '{print $6;}')
POD2_IP=$(kubectl get pods -o wide --no-headers=true | tail -1 | awk '{print $6;}')

echo POD1=$POD1 POD1_IP=$POD1_IP
echo POD2=$POD2 POD2_IP=$POD2_IP

POD1_IP=10.192.3.1
POD2_IP=10.192.2.1


Let's look at the yaml information for POD1:

In [69]:
kubectl get pod $POD1 -o yaml

apiVersion: v1
items:
- apiVersion: v1
  kind: Pod
  metadata:
    annotations:
      kubernetes.io/created-by: |
        {"kind":"SerializedReference","apiVersion":"v1","reference":{"kind":"ReplicaSet","namespace":"default","name":"k8s-demo-7bc74f7946","uid":"aa02aead-fc92-11e7-8b3e-02420ac00002","apiVersion":"extensions","resourceVersion":"27297"}}
    creationTimestamp: 2018-01-18T21:00:50Z
    generateName: k8s-demo-7bc74f7946-
    labels:
      app: k8s-demo
      pod-template-hash: "3673093502"
    name: k8s-demo-7bc74f7946-8nmlb
    namespace: default
    ownerReferences:
    - apiVersion: extensions/v1beta1
      blockOwnerDeletion: true
      controller: true
      kind: ReplicaSet
      name: k8s-demo-7bc74f7946
      uid: aa02aead-fc92-11e7-8b3e-02420ac00002
    resourceVersion: "27331"
    selfLink: /api/v1/namespaces/default/pods/k8s-demo-7bc74f7946-8nmlb
    uid: aa0683e6-fc92-11e7-8b3e-02420ac00002
  spec:
    containers:
    - image: mjbright/k8s-demo:1
      imagePullP

So now we can access to the pods directly on their port 8080:

In [10]:
curl http://$POD1_IP:8080


                ./shhs:`                
            .+ymmdyosydNms/`            
        -+yNmhs+++++++++oydNmy/.        
    .+hNmhs++++++++yy+++++++oydNmy+.    
   +Mds++++++++++++md++++++++++++smM/   
  `MN++++++++++oshhNNhys+++++++++++Nm   
  oMy++++mmsohNMNddMMhdNMmy+ymm++++yM/  
  NN++++++omMMmo++sMM+++sNMMdo++++++Nd  
 /Mh++++++yMNdMNhsyMMssdMMdNMs++++++hM- 
 mMo++++++NMo+ohMMMNNMMNh++sMm++++++oMy 
:Md++++++oMNosydNMm++NMNhysoMM+++++++hM.
hMo+++shhdMMNNmddMMNNMNddmNMMMdhyo+++oMs
mMo+++syo+sMmo++sMMhdMNo++oNMy+oss+++oMh
-mMy+++++++sNNysMMy++hMNohMNs+++++++yNh`
  oMmo+++++++ymMMmhyyhNMMmy+++++++omm:  
   .hMh+++++++hNyyhddhyymh+++++++hMo`   
     /NNs++++yNo++++++++oNy++++sNd.     
      `sMdo++++++++++++++++++odN/       
        -dMhsssssssssssssssshNy`        
          :shhhhhhhhhhhhhhhyo.          


Served from container k8s-demo-7bc74f7946-8nmlb


In [11]:
curl http://$POD2_IP:8080


                ./shhs:`                
            .+ymmdyosydNms/`            
        -+yNmhs+++++++++oydNmy/.        
    .+hNmhs++++++++yy+++++++oydNmy+.    
   +Mds++++++++++++md++++++++++++smM/   
  `MN++++++++++oshhNNhys+++++++++++Nm   
  oMy++++mmsohNMNddMMhdNMmy+ymm++++yM/  
  NN++++++omMMmo++sMM+++sNMMdo++++++Nd  
 /Mh++++++yMNdMNhsyMMssdMMdNMs++++++hM- 
 mMo++++++NMo+ohMMMNNMMNh++sMm++++++oMy 
:Md++++++oMNosydNMm++NMNhysoMM+++++++hM.
hMo+++shhdMMNNmddMMNNMNddmNMMMdhyo+++oMs
mMo+++syo+sMmo++sMMhdMNo++oNMy+oss+++oMh
-mMy+++++++sNNysMMy++hMNohMNs+++++++yNh`
  oMmo+++++++ymMMmhyyhNMMmy+++++++omm:  
   .hMh+++++++hNyyhddhyymh+++++++hMo`   
     /NNs++++yNo++++++++oNy++++sNd.     
      `sMdo++++++++++++++++++odN/       
        -dMhsssssssssssssssshNy`        
          :shhhhhhhhhhhhhhhyo.          


Served from container k8s-demo-7bc74f7946-mmc98


# Kube-proxy

## TODO: Accessing our application - accessing via kube-proxy

<hr/>